In [16]:
# ! git clone https://github.com/haje01/gym-tictactoe.git
# %cd gym-tictactoe
# ! pip install -e ./

In [1]:
import numpy as np
from gym_tictactoe.env import TicTacToeEnv
import gym
import random
import pickle

In [14]:
env = TicTacToeEnv(gym.Env)

action_space = env.action_space
observable_space = env.observation_space.n

q_table = np.zeros([9,9,9,9,9,9,9,9,9])
q_table_X = np.zeros([9,9,9,9,9,9,9,9,9])

In [3]:
alpha = 0.6 # Learning rate #
gamma = 0.6 # Preference for short term reward #
epsilon = 0.4 # Preference for exploitation vs exploration #
decay_rate= 0.005 # How much it changes to prefer exploitation #

episodes = 1 # Number of games played #

In [12]:
def TicTacToe_Match_Rand(alpha, gamma, epsilon, decay_rate, episodes, q_table):
  rewards, epochs, total_penalties = 0, 0, 0

  for replay in range(episodes):
      state = env.reset() # Resets enviroment at the beggining of a game #
      done = False
      epochs += 1
      env.mark = 'X'

      while True:
          reward = 0
          board = state[0]

          if random.uniform(0, 1) < epsilon or env.mark == 'X': # Explore #
              action = random.choice(env.available_actions())
          else: # Exploit #
                action = np.argmax(q_table[board]).round()
                if board[action] > 0:
                  action = random.choice(env.available_actions())

          new_state, reward, done, info = env.step(action) # New variables created as action is executed #
          new_board = new_state[0]

          rewards += reward
          
          q_table[board,action] = q_table[board,action] + alpha * (reward + gamma * np.max(q_table[new_board,:])-q_table[board,action])

          board = new_state[0] # Updates enviroment #
          state = new_state
          epsilon = np.exp(-decay_rate * epochs) # Decays epsilon so AI prefers exploitation more over time #

          print(f"Total score: {rewards}")
          print(f"Score for round: {reward}")
          print(f"Number of games: {epochs}")
          print("\n")
          env.render()

          if done == True:
            break
  with open('Q-Tic-Tac-Toe.pkl', 'wb') as file:
    pickle.dump(q_table, file)

In [10]:
def TicTacToe_Match_AI(alpha, gamma, epsilon, decay_rate, episodes, q_table, q_table_X):
  rewards, epochs, total_penalties, X_rewards = 0, 0, 0, 0

  for replay in range(episodes):
      state = env.reset() # Resets enviroment at the beggining of a game #
      done = False
      epochs += 1
      env.mark = 'X'

      while True:
          reward = 0
          board = state[0]

          if random.uniform(0, 1) < epsilon: # Explore #
              action = random.choice(env.available_actions())
          else: # Exploit #
                if env.mark == 'X':
                  action = np.argmax(q_table[board]).round()
                else:
                  action = np.argmax(q_table_X[board]).round()

                if board[action] > 0:
                  action = random.choice(env.available_actions())

          new_state, reward, done, info = env.step(action) # New variables created as action is executed #
          new_board = new_state[0]

          rewards += reward
          X_rewards += -reward
          
          q_table[board,action] = q_table[board,action] + alpha * (reward + gamma * np.max(q_table[new_board,:])-q_table[board,action])
          q_table_X[board,action] = q_table_X[board,action] + alpha * (-reward + gamma * np.max(q_table_X[new_board,:])-q_table_X[board,action])


          board = new_state[0] # Updates enviroment #
          state = new_state
          epsilon = np.exp(-decay_rate * epochs) # Decays epsilon so AI prefers exploitation more over time #

          print(f"O's score: {rewards}")
          print(f"X's score: {X_rewards}")
          print(f"Number of games: {epochs}")
          print("\n")
          env.render()

          if done == True:
            break
  with open('Q-Tic-Tac-Toe.pkl', 'wb') as file:
    pickle.dump(q_table, file)
  with open('Q-Tic-Tac-Toe_X.pkl', 'wb') as file:
    pickle.dump(q_table_X, file)